In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("List available databases and tables").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.catalog.listDatabases()

[Database(name='2022_10_22', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='2023_04_01', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse')]

In [2]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark.sql("USE 2023_04_01")
    # Assuming that 'ticker' is a valid Python variable

# query = f"""SELECT ticker_region FROM sym_ticker_region WHERE ticker_region LIKE "%-US" """
# df = spark.sql(query)
# df = df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
# ticker_list = df.collect()
# print(len(ticker_list))


DataFrame[]

In [85]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import when
import pyspark.pandas as ps
import numpy as np


#fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))

def query(ticker,date):
    query = f"""SELECT d.ticker_region, a.date, FF_PRICE_CLOSE_FP
                FROM FF_BASIC_AF a 
                LEFT JOIN sym_ticker_region d ON d.fsym_id = a.fsym_id 
                WHERE d.ticker_region = "{ticker}-US" AND a.date = "{date}"
                """

    fund_df = spark.sql(query)
    fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
    
    return fund_df

def pull_prev_data(df, metric):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    metric_curr = f'{metric}_curr'
    metric_prev = f'{metric}_prev'
    query = f"""
                SELECT t.Ticker, t.Implosion_Date, t.Implosion_Year, t.Implosion_Prev4Years, s.ticker_region, a.{metric} AS {metric_curr},
                       b.{metric} AS {metric_prev}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON t.Ticker = SUBSTRING(s.ticker_region, 1, LENGTH(s.ticker_region)-3) AND SUBSTRING(s.ticker_region, -2, 3) = 'US'
                LEFT JOIN FF_BASIC_AF a ON s.fsym_id = a.fsym_id AND YEAR(a.date) = t.Implosion_Year
                LEFT JOIN FF_BASIC_AF b ON s.fsym_id = b.fsym_id AND YEAR(b.date) = t.Implosion_Prev4Years
                ORDER BY t.Ticker
            """
    result_df_spark = spark.sql(query)
    df = ps.DataFrame(result_df_spark)
    
    df['pct_change'] = (df[metric_curr] - df[metric_prev])/df[metric_prev]
    df['pct_change'] = df['pct_change'].replace([np.inf, -np.inf], np.nan) 
    df=df.dropna(axis=0)
    mean_val = df['pct_change'].mean()
    stddev_val = df['pct_change'].std()
    z_score_threshold = 3.0
    df = df[
    (df['pct_change'] >= mean_val - z_score_threshold * stddev_val) &
    (df['pct_change'] <= mean_val + z_score_threshold * stddev_val)]
    new_mean = df['pct_change'].mean()
    return new_mean


def create_df(filename):
    df = pd.read_csv(filename, index_col=False)
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date']).dt.date
    df['Implosion_Year'] = df['Implosion_Date']
    df['Implosion_Year'] =  pd.to_datetime(df['Implosion_Year']).dt.year
    df['Implosion_Prev4Years'] = df['Implosion_Date'] - pd.DateOffset(years=4)
    df['Implosion_Prev4Years'] =  pd.to_datetime(df['Implosion_Prev4Years']).dt.year
    df = df.sort_values(by='Ticker')
    df_metrics = spark.sql("SELECT * FROM FF_BASIC_AF LIMIT 10")
    df_metrics = df_metrics.columns
    print(len(df_metrics))
    df_metrics = df_metrics[6:]
    df_metrics.remove('ff_actg_standard')
    metric_dict = {}
    for m in df_metrics:
        print(m)
        avg = pull_prev_data(df, m)
        metric_dict[m] = avg
    # df=pull_prev_data(df, 'FF_PRICE_CLOSE_FP')
    # return df
    metric_df = pd.DataFrame(list(metric_dict.items()), columns=['Metric', 'Value'])
    metric_df.to_csv('ChangesBeforeImplosion.csv', index=False)
    return metric_df
    

#df = create_df('imploded_tickers_dates.csv')
# df.show(100)
# rdf=query('AAPL','2012-01-01')
# rdf.show()

In [ ]:
#spark.stop()

In [83]:
def get_top_bottom_five(df):
    df = df.sort_values(by='Value')
    df=df.dropna()
    top5 = df.head(5)
    down5 = df.tail(5)
    print(top5)
    print(down5)


df = pd.read_csv('ChangesBeforeImplosion.csv')
get_top_bottom_five(df)

            Metric      Value
3     ff_gross_inc -32.281477
40       ff_com_eq -28.976753
43    ff_shldrs_eq -25.431693
13      ff_inc_tax  -6.418365
14  ff_min_int_exp  -1.021970
            Metric      Value
25  ff_assets_curr  41.846761
31      ff_debt_st  42.718225
41         ff_debt  45.450333
18      ff_cash_st  57.866520
30   ff_liabs_curr  65.506074


In [77]:
metric_df

NameError: name 'metric_df' is not defined